In [2]:
!wget --header="Host: storage.googleapis.com" --header="User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.129 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9" --header="Accept-Language: en-IN,en;q=0.9,en-US;q=0.8,te;q=0.7" --header="Referer: https://www.kaggle.com/" "https://storage.googleapis.com/kaggle-data-sets/623/1190/bundle/archive.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1588590612&Signature=rheWuPjPmNOLvUrlUIvvZX1pqg1ZKDfxYvKVeqxTH8qOYuTjeltFvDpJ7E2kpsGoNXsstY2%2F9i292rpSBHTd%2BHWLxW3ZLgHEOg62EqGrG15iwT1MdAPjcKFQcu0CzILrtclPlyHGsm4Z1Kb01HO%2FntP9qG6Nusi98PpfxG65tT8W%2FQPwASIIJc1mRL1GZVM91ZrfADBfxDQ13BY4qt%2F952a1sJkfJoDXUkRNiL3kPyx64sjaiRu3QC59DYG%2B%2BYYtHlM3Q1CJKbakOKJdiwcUzFLu75er6xQivhbaVkrhMCP6mONNRh6rM1sHxr%2FHWD1wf4eqbEfahLsuuG5Gj%2BM%2Fjw%3D%3D&response-content-disposition=attachment%3B+filename%3Dqa-jokes.zip" -c -O 'qa-jokes.zip'

--2020-05-01 12:31:40--  https://storage.googleapis.com/kaggle-data-sets/623/1190/bundle/archive.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1588590612&Signature=rheWuPjPmNOLvUrlUIvvZX1pqg1ZKDfxYvKVeqxTH8qOYuTjeltFvDpJ7E2kpsGoNXsstY2%2F9i292rpSBHTd%2BHWLxW3ZLgHEOg62EqGrG15iwT1MdAPjcKFQcu0CzILrtclPlyHGsm4Z1Kb01HO%2FntP9qG6Nusi98PpfxG65tT8W%2FQPwASIIJc1mRL1GZVM91ZrfADBfxDQ13BY4qt%2F952a1sJkfJoDXUkRNiL3kPyx64sjaiRu3QC59DYG%2B%2BYYtHlM3Q1CJKbakOKJdiwcUzFLu75er6xQivhbaVkrhMCP6mONNRh6rM1sHxr%2FHWD1wf4eqbEfahLsuuG5Gj%2BM%2Fjw%3D%3D&response-content-disposition=attachment%3B+filename%3Dqa-jokes.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.13.80, 2607:f8b0:4004:811::2010
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.13.80|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1423390 (1.4M) [application/zip]
Saving to: ‘qa-jokes.zip’

qa-jokes.zip        100%[===================>]   1.36M  --.-K

In [4]:
from zipfile import ZipFile 
  
# specifying the zip file name 
file_name = "qa-jokes.zip"
  
# opening the zip file in READ mode 
with ZipFile(file_name, 'r') as zip: 
    # printing all the contents of the zip file 
    zip.printdir() 
  
    # extracting all the files 
    print('Extracting all the files now...') 
    zip.extractall() 
    print('Done!') 
del zip

File Name                                             Modified             Size
jokes.csv                                      2019-09-20 04:54:38      3508579
Extracting all the files now...
Done!


In [0]:
import pandas as pd
data=pd.read_csv('jokes.csv',nrows=15000)

In [6]:
data.head()

,ID,Question,Answer
0,1,Did you hear about the Native American man tha...,He nearly drown in his own tea pee.
1,2,What's the best anti diarrheal prescription?,Mycheexarphlexin
2,3,What do you call a person who is outside a doo...,Matt
3,4,Which Star Trek character is a member of the m...,Jean-Luc Pickacard
4,5,What's the difference between a bullet and a h...,A bullet doesn't miss Harambe


In [0]:
import warnings
warnings.filterwarnings("ignore")
import tensorflow as tf
tf.compat.v1.enable_eager_execution()
import os
import re
path=os.getcwd()+'/jokes.csv'
# To read the file
import io
import numpy as np
# For normalizing data
import unicodedata
# Tensorflow libraries
from tensorflow.keras.preprocessing.text import Tokenizer 
from tensorflow.keras.preprocessing.sequence import pad_sequences 
from tensorflow.keras.layers import Input, Softmax, RNN, Dense, Embedding, LSTM,TimeDistributed
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
# Metric that can tell the strength of relation between two sentences
import nltk.translate.bleu_score as bleu

In [8]:
data.head()

,ID,Question,Answer
0,1,Did you hear about the Native American man tha...,He nearly drown in his own tea pee.
1,2,What's the best anti diarrheal prescription?,Mycheexarphlexin
2,3,What do you call a person who is outside a doo...,Matt
3,4,Which Star Trek character is a member of the m...,Jean-Luc Pickacard
4,5,What's the difference between a bullet and a h...,A bullet doesn't miss Harambe


In [0]:
# Converting into unicode --http://stackoverflow.com/a/518232/2809427
def unicode_to_ascii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn')

# Function to convert into lower case, remove punctuation.
def normalize_string(s):
    s = unicode_to_ascii(s.lower().strip())
    s = re.sub(r'([!.?])', r' \1', s)
    s = re.sub(r'[^a-zA-Z.!?]+', r' ', s)
    s = re.sub(r'\s+', r' ', s)
    return s

In [0]:
data['question_preprocess']=data['Question'].apply(lambda x: normalize_string(x))

In [0]:
data['answer_preprocess']=data['Answer'].apply(lambda x : normalize_string(x))

In [0]:
data['answer_preprocess_in']=data['answer_preprocess'].apply(lambda x : '<start> '+x)

In [0]:
data['answer_preprocess_out']=data['answer_preprocess'].apply(lambda x: x+' <end>')

In [0]:
data['answer_preprocess']=data['Answer'].apply(lambda x : '<start> '+ x + ' <end>')

In [15]:
data.head()

,ID,Question,Answer,question_preprocess,answer_preprocess,answer_preprocess_in,answer_preprocess_out
0,1,Did you hear about the Native American man tha...,He nearly drown in his own tea pee.,did you hear about the native american man tha...,<start> He nearly drown in his own tea pee. <end>,<start> he nearly drown in his own tea pee .,he nearly drown in his own tea pee . <end>
1,2,What's the best anti diarrheal prescription?,Mycheexarphlexin,what s the best anti diarrheal prescription ?,<start> Mycheexarphlexin <end>,<start> mycheexarphlexin,mycheexarphlexin <end>
2,3,What do you call a person who is outside a doo...,Matt,what do you call a person who is outside a doo...,<start> Matt <end>,<start> matt,matt <end>
3,4,Which Star Trek character is a member of the m...,Jean-Luc Pickacard,which star trek character is a member of the m...,<start> Jean-Luc Pickacard <end>,<start> jean luc pickacard,jean luc pickacard <end>
4,5,What's the difference between a bullet and a h...,A bullet doesn't miss Harambe,what s the difference between a bullet and a h...,<start> A bullet doesn't miss Harambe <end>,<start> a bullet doesn t miss harambe,a bullet doesn t miss harambe <end>


In [0]:
ques_tokenizer=Tokenizer(filters='')
ques_tokenizer.fit_on_texts(data['question_preprocess'])
# inp_lan will have an integer represented for each word
inp_lan=ques_tokenizer.texts_to_sequences(data['question_preprocess'])
# Pad sequences (By default it will pad zeros till the max length of the ita_data)
inp_lan=pad_sequences(inp_lan,padding='post')
# Here inp_lan is the encoder input data.

In [0]:
# Declare the tokenizer for english words
ans_tokenizer=Tokenizer(filters='')
# we are fitting on eng_data not on eng_in or eng_out as it has both <start> and <end> tags
ans_tokenizer.fit_on_texts(data['answer_preprocess'])

In [0]:
# Decoder input data
tar_lan_in=ans_tokenizer.texts_to_sequences(data['answer_preprocess_in'])
tar_lan_in=pad_sequences(tar_lan_in,padding='post')

In [0]:
# Decoder output data
tar_lan_out=ans_tokenizer.texts_to_sequences(data['answer_preprocess_out'])
tar_lan_out=pad_sequences(tar_lan_out,padding='post')


In [0]:
# Max length of input_sequence, max_length of target sequence 
inp_seq_length=inp_lan[0].shape[0]
tar_seq_length=tar_lan_in[0].shape[0]

In [0]:
# Total number of unique words in Italian sentence 
ques_vocab_size=len(ques_tokenizer.word_index)+1
# Total number of unique words in english sentence
ans_vocab_size=len(ans_tokenizer.word_index)+1

In [0]:
# Max length of input_sequence, max_length of target sequence 
inp_seq_length=inp_lan[0].shape[0]
tar_seq_length=tar_lan_in[0].shape[0]

ans_index_word={}
ans_word_index={}
for key,value in ans_tokenizer.word_index.items():
  ans_index_word[value]=key 
  ans_word_index[key]=value



In [23]:
ans_word_index

{'<start>': 1,
 '<end>': 2,
 'a': 3,
 'the': 4,
 'to': 5,
 'because': 6,
 'you': 7,
 'they': 8,
 'he': 9,
 'it': 10,
 'and': 11,
 'of': 12,
 'in': 13,
 'i': 14,
 'was': 15,
 'is': 16,
 'on': 17,
 'his': 18,
 'one': 19,
 "it's": 20,
 'for': 21,
 'have': 22,
 'your': 23,
 'with': 24,
 'are': 25,
 'get': 26,
 'that': 27,
 'my': 28,
 "don't": 29,
 'just': 30,
 'an': 31,
 'can': 32,
 'out': 33,
 'be': 34,
 'when': 35,
 'all': 36,
 'both': 37,
 'but': 38,
 'she': 39,
 'at': 40,
 'not': 41,
 'if': 42,
 'their': 43,
 "they're": 44,
 'no': 45,
 'up': 46,
 'this': 47,
 'her': 48,
 'so': 49,
 'has': 50,
 'like': 51,
 'them': 52,
 'do': 53,
 "can't": 54,
 'what': 55,
 'other': 56,
 'had': 57,
 'by': 58,
 "i'm": 59,
 'from': 60,
 'it.': 61,
 'me': 62,
 'only': 63,
 'always': 64,
 "he's": 65,
 'got': 66,
 'go': 67,
 'will': 68,
 'never': 69,
 'how': 70,
 'about': 71,
 'there': 72,
 'would': 73,
 'him': 74,
 "doesn't": 75,
 'know': 76,
 'too': 77,
 'little': 78,
 'make': 79,
 'were': 80,
 'people': 8

In [24]:
print('input lang in',inp_lan.shape)
print('target lang in',tar_lan_in.shape)
print('target lang out',tar_lan_out.shape)

# Total number of unique words in Italian sentence 
ques_vocab_size=len(ques_tokenizer.word_index)+1
# Total number of unique words in english sentence
ans_vocab_size=len(ans_tokenizer.word_index)+1

print('question vocab size',ques_vocab_size)
print('answer vocab size',ans_vocab_size)
#print(ita_tokenizer_in.word_index)

input lang in (15000, 26)
target lang in (15000, 112)
target lang out (15000, 112)
question vocab size 10672
answer vocab size 18979


In [0]:
#Encoder input shape will be (Batch_size,inp_seq_length) --
input_encoder=Input(shape=(inp_seq_length,))

# Encoder LSTM needs the input to be 3 dim. One easy way to add a dimension to this integer sequences is to add a Embedding layer.

# input_encoder is passed to the embedding layer.  
encoder_embedding=Embedding(input_dim=ques_vocab_size,output_dim=128)(input_encoder)
# encoder_embedding shape is (Batch_size,inp_seq_length,output_dim) --
# Trainable paramters is ita_vocab_size * 128. 
# Each integer in the input_encoder is converted to 128 dimensional vector


encoder_lstm=LSTM(256,return_state=True)(encoder_embedding)
# Encoder_LSTM will return three arrays 
# 1. hidden_state/output of the last time step (None,256), 
# 2. hidden_state/ output of the last time step(None,256), 
# 2. cell_state of the last time step (None,256)


In [0]:
encoder_states=encoder_lstm[1:]
# Select the last two hidden states.

In [0]:
# Decoder input, we will keep the shape (None). Note: If none it can accept any size of data in dimension
# The reason we are not fixing the input_decoder to max_length is during 
# inference we will be sending only one input at a time step and predict. 

input_decoder = Input(shape=(None,))
# input_decoder shape is (None,None)

decoder_embedding_layer=Embedding(input_dim=ans_vocab_size,output_dim=128)
decoder_embedding_output=decoder_embedding_layer(input_decoder)
# decoder_embedding_layer - (None,None,128) 

decoder_lstm=LSTM(256,return_sequences=True,return_state=True)
decoder_output,_,_=decoder_lstm(decoder_embedding_output,initial_state=encoder_states)
# decoder_lstm will return three outputs
# 1. All the hiddenstate/output at each time step (None,None,256)
# 2. hiddenstate/output at last time step (None,256)
# 3. Cell state at last time step (None,256)


In [0]:
# Apply dense layer(units=eng_vocab_size) at every time step of the decoder_output
decoder_dense=TimeDistributed( Dense(ans_vocab_size,activation='softmax'))
dense_output= decoder_dense(decoder_output)
# dense_output shape (None,none, eng_vocab_size)


In [0]:
# Our model has two inputs, one to the encoder and other to the decoder. 
# outputs - Dense layer output
model=Model([input_encoder,input_decoder],dense_output)

In [30]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 26)]         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 26, 128)      1366016     input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 128)    2429312     input_2[0][0]                    
______________________________________________________________________________________________

In [31]:
# if input_decoder shape is (None,max_length)
# Dense output shape is (None,max_length,eng_vocab_size), decoder_output (None,max_length)
# Target data is just integers, while predicted data is a softmax output
# we apply sparse_categorical_crossentropy instead of categorical_crossentropy
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')
# Train the model
model.fit([inp_lan, tar_lan_in], tar_lan_out,
          batch_size=64,
          epochs=50,
          validation_split=0.2)  

Epoch 1/50
188/188 [==============================] - 58s 310ms/step - loss: 0.7472 - val_loss: 0.4054
Epoch 2/50
188/188 [==============================] - 58s 306ms/step - loss: 0.3916 - val_loss: 0.3855
Epoch 3/50
188/188 [==============================] - 58s 306ms/step - loss: 0.3734 - val_loss: 0.3732
Epoch 4/50
188/188 [==============================] - 58s 307ms/step - loss: 0.3595 - val_loss: 0.3657
Epoch 5/50
188/188 [==============================] - 58s 307ms/step - loss: 0.3492 - val_loss: 0.3616
Epoch 6/50
188/188 [==============================] - 58s 307ms/step - loss: 0.3414 - val_loss: 0.3579
Epoch 7/50
188/188 [==============================] - 58s 306ms/step - loss: 0.3335 - val_loss: 0.3550
Epoch 8/50
188/188 [==============================] - 58s 306ms/step - loss: 0.3266 - val_loss: 0.3534
Epoch 9/50
188/188 [==============================] - 58s 306ms/step - loss: 0.3203 - val_loss: 0.3529
Epoch 10/50
188/188 [==============================] - 58s 307ms/step - l

In [43]:
model.fit([inp_lan, tar_lan_in], tar_lan_out,
          batch_size=64,
          epochs=20,
          validation_split=0.2)  

Epoch 1/20
188/188 [==============================] - 58s 306ms/step - loss: 0.1172 - val_loss: 0.4398
Epoch 2/20
188/188 [==============================] - 58s 306ms/step - loss: 0.1152 - val_loss: 0.4421
Epoch 3/20
188/188 [==============================] - 58s 307ms/step - loss: 0.1128 - val_loss: 0.4428
Epoch 4/20
188/188 [==============================] - 57s 306ms/step - loss: 0.1111 - val_loss: 0.4444
Epoch 5/20
188/188 [==============================] - 57s 305ms/step - loss: 0.1089 - val_loss: 0.4462
Epoch 6/20
188/188 [==============================] - 57s 305ms/step - loss: 0.1067 - val_loss: 0.4487
Epoch 7/20
188/188 [==============================] - 57s 306ms/step - loss: 0.1049 - val_loss: 0.4494
Epoch 8/20
188/188 [==============================] - 57s 305ms/step - loss: 0.1029 - val_loss: 0.4524
Epoch 9/20
188/188 [==============================] - 57s 305ms/step - loss: 0.1012 - val_loss: 0.4539
Epoch 10/20
188/188 [==============================] - 57s 305ms/step - l

In [0]:
#Encoder model for inference
encoder_model=Model(input_encoder,encoder_lstm)
# encoder_model takes a integer sequence of Italian language as input and returns hidden_state and cell_state

In [0]:
# Decoder states - shape is LSTM_Size
decoder_state_h=Input(shape=(256))
decoder_state_c=Input(shape=(256))
decoder_state_inputs=[decoder_state_h,decoder_state_c]

In [0]:
# Input to decoder is send to embedding layer
decoder_embedding_output=decoder_embedding_layer(input_decoder)

# Get the decoder outputs when a particular input and initial_state is given
decoder_outputs,state_h,state_c=decoder_lstm(decoder_embedding_output,initial_state=decoder_state_inputs)
decoder_states=[state_h,state_c]
# Send the decoder outputs to the dense layer to get the predicted word
decoder_outputs=decoder_dense(decoder_outputs)


In [0]:
# decoder model takes two inputs 1. Decoder input 2. Decoder initial states
# returns two outputs 1. Decoder outputs 2. decoder states(hidden_state and cell state)
decoder_model=Model([input_decoder]+decoder_state_inputs,[decoder_outputs]+decoder_states)

In [0]:
from prettytable import PrettyTable
    
x = PrettyTable()
x.field_names = ["Input data", "Actual data", "Predicted data"]

In [0]:
def inference(seq):
  '''
    Function that takes Italian sequence and returns English sequence
  '''
  # Input the sequence to encoder_model and get the final timestep encoder_states(Hidden and cell state)
  seq=seq.reshape(-1,inp_seq_length)
  encoder_state_value=encoder_model.predict(seq)
  encoder_state_value=encoder_state_value[1:]
  # encoder_state_value[0] -- hidden_state -- shape (None,256)
  # encoder_state_value[1] -- cell_state -- shape (None,256)

  # Target word
  target_word=np.zeros((1,1))
  # <start>:1 , <end>:2 -- 
  target_word[0,0]=1
  
  stop_condition=False
  sent=''
  k=0
  while not stop_condition:

      # We are giving a target_word which represents <start> and encoder_states to the decoder_model
      output,state_h,state_c=decoder_model.predict([target_word]+encoder_state_value)
      # As the target word length is 1. We will only have one time step
      encoder_state_value=[state_h,state_c]
      # Output shape (1,eng_vocab_size). Find the word which the decoder predicted with max_probability
      output=np.argmax(output,-1)
      #print(output)
      # The output is a integer sequence, to get back the word. We use our lookup table reverse_dict
      sent=sent+' '+ans_index_word[int(output)]
      k+=1
      # If the max_length of the sequence is reached or the model predicted 2 (<end>) stop your model
      if k>tar_seq_length or output==2:
        stop_condition=True
      target_word=output.reshape(1,1)
  
  return sent





In [0]:
import random

In [70]:
from prettytable import PrettyTable
    
x = PrettyTable()
x.field_names = ["Input data", "Actual data", "Predicted data"]
kk=[]
for i in range(3):
  index=random.randint(1,1000)
  pred=inference(inp_lan[index])
  kk.append(index)
  x.add_row([data['Question'][index],data['Answer'][index],pred])
  # print('Input data is: ',ita_data[index])
  # print('Actual data is: ',eng_out[index])
  # print('Predicted data is:',pred)
print(x)

+-----------------------------------------------------------+------------------------------------+----------------------------------+
|                         Input data                        |            Actual data             |          Predicted data          |
+-----------------------------------------------------------+------------------------------------+----------------------------------+
|             Which president was least guilty?             | Lincoln, because he is in a cent.  |  because he is in a cent . <end> |
| Whats the name of a bodybuilder whose a fan of the X-Men? |            Huge Jackman            |        huge they re <end>        |
|  Why does Willem Dafoe play a villain in a lot of movies? |      Duh.  Cause he's da foe.      |              . <end>             |
+-----------------------------------------------------------+------------------------------------+----------------------------------+


In [67]:
inp_lan[10]

array([   8,   31,   96,   52,   39,  671, 2525,   13,  241, 3907,    1,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0], dtype=int32)

In [0]:
data['Question'][10]

'Why are there so many blood cells in female prisons?'